# Resource Allocation Code Demo

The Food Bank of the Southern Tier (FBST) is a member of Feeding America, focused on providing food security for people with limited financial resources, and serves six counties and nearly 4,000 square miles in the New York.  Under normal operations (non COVID times), the Mobile Food Pantry program is among the main activities of the FBST.  The goal of the service is to make nutritious and healthy food more accessible to people in underserved communities.  Even in areas where other agencies provide assistance, clients may not always have access to food due to limited public transportation options, or because those agencies are only open hours or days per work.

Here we do a sample experiment testing out some of the existing and developed algorithms against a randomized heuristic.

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

In [2]:
# Getting out configuration parameter for the environment
CONFIG = or_suite.envs.env_configs.resource_allocation_foodbank_config(3)
# CONFIG = or_suite.envs.env_configs.resource_allocation_default_config


# Specifying training iteration, epLen, number of episodes, and number of iterations
epLen = CONFIG['num_rounds']
nEps = 1
numIters = 2

# Configuration parameters for running the experiment
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/resource/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, # save trajectory for calculating additional metrics
                    'epLen' : epLen,
                    'render': False,
                    'pickle': False # indicator for pickling final information
                    }

resource_env = gym.make('Resource-v0', config=CONFIG)
mon_env = Monitor(resource_env)


In [3]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
#  'Random': or_suite.agents.rl.random.randomAgent(),
#  'Equal': or_suite.agents.resource_allocation.equal_allocation.equalAllocationAgent(epLen, CONFIG),
#  'FixedThreshold': or_suite.agents.resource_allocation.fixed_threshold.fixedThresholdAgent(epLen, CONFIG),
 'Guardrail-0.5': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.5),
#  'Guardrail-0.3': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.3),
#  'Guardrail-0.25': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.25)
}

Mean and variance endomwnets:
[[36.66488446 50.41276469 36.34946901]
 [43.95244355 60.70726574 43.73277552]
 [65.78668211 91.91349249 65.44240092]] [[  87.06560246  155.48197986  589.03745581]
 [ 174.328777    329.57167123  929.44619453]
 [ 795.64847456 1694.07540568 2888.09737205]]
Mean and variance endomwnets:
[[40.4304571   8.6044611  35.25868755]
 [48.67806343 10.52719357 42.21112546]
 [71.95769637 15.92069786 63.86573431]] [[  91.11649898    4.5873471   610.49597968]
 [ 205.52798238    9.7622034   952.33464102]
 [1026.29245453   50.02667417 2671.09559557]]
Mean and variance endomwnets:
[[ 66.86340481  14.03271871  34.35091664]
 [ 80.33388422  17.03301621  41.81046881]
 [120.72047828  26.21419851  62.10243205]] [[ 273.10377418   11.55578419  549.15967234]
 [ 548.18569278   26.68250428  887.71970428]
 [2859.27772253  132.96503133 2576.1310948 ]]


# Step 5: Run Simulations

Run the different heuristics in the environment

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/resource_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(resource_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(resource_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/resource_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/resource_'+str(agent)+'/')
        algo_list_radar.append(str(agent))     
        
fig_path = '../figures/'
fig_name = 'resource'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)        
        
additional_metric = { 'Efficiency': lambda traj : or_suite.utils.delta_EFFICIENCY(traj, CONFIG),
                    'Hindsight Envy': lambda traj : or_suite.utils.delta_HINDSIGHT_ENVY(traj, CONFIG),
                      'Counterfactual Envy': lambda traj : or_suite.utils.delta_COUNTERFACTUAL_ENVY(traj, CONFIG),
                    'Budget': lambda traj : or_suite.utils.times_out_of_budget(traj, CONFIG)}
#                       'Prop': lambda traj : or_suite.utils.delta_PROP(traj, CONFIG), \
#                       'Exante Envy': lambda traj : or_suite.utils.delta_EXANTE_ENVY(traj, CONFIG)}
fig_name = 'resource'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

Guardrail-0.5

env reset!
starting_state [519.2      519.2      519.2      519.2      519.2       61.867798
  43.10978   60.375957]
Lower and Upper Solutions:
[[0.20146 0.21181 0.36877 0.21525 0.     ]
 [0.32994 0.30984 0.      0.30317 0.     ]
 [0.13981 0.14636 0.24919 0.14853 0.50092]]
[[0.47806 0.50202 0.86896 0.50996 0.     ]
 [0.78127 0.73279 0.      0.71674 0.     ]
 [0.32941 0.3459  0.592   0.35136 1.18518]]
timestep:  0
new state [436.0547   435.6667   429.67523  435.53772  488.904     41.256927
  75.95979    5.776585]
reward:  -0.6129640600744122
timestep:  1
new state [355.08344  357.2942   390.36688  358.02527  481.99908   48.038857
  56.942787  82.43221 ]
reward:  -0.41037753218464124
timestep:  2
new state [437.4      437.4      437.4      437.4      437.4       22.08356
  26.83384   50.112766]
reward:  -0.5395880389906147

env reset!
starting_state [437.4       437.4       437.4       437.4       437.4         8.275917
   6.1462727  23.933739 ]
timestep:  0
new state [420

In [6]:
from IPython.display import IFrame
IFrame("../figures/resource_line_plot.pdf", width=600, height=280)

In [7]:
IFrame("../figures/resource_radar_plot.pdf", width=600, height=500)